Date: --

Goal: Walk step-by-step thru `ForAndy.py`, which creates train/test splits by sites (originally created by Krishna Rao, adapted by Kuai Fang).
- Figure out how to extract sites for train/test split for each fold for transformer implementation

In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [15]:
data_file='/Users/andyhuynh/Documents/lfmc/geolearn/app/vegetation/attention/andy/playground/recreate_train_test/lstm_input_data_pure+all_same_28_may_2019_res_SM_gap_3M'

int_lag=6 # UNSURE
dataset= pd.read_pickle(data_file)

DROPCROPS = True
lc_dict = {14: 'crop',
            20: 'crop',
            30: 'crop',
            50: 'closed broadleaf deciduous',
            70: 'closed needleleaf evergreen',
            90: 'mixed forest',
            100:'mixed forest',
            110:'shrub/grassland',
            120:'grassland/shrubland',
            130:'closed to open shrub',
            140:'grass',
            150:'sparse vegetation',
            160:'regularly flooded forest'}
TRAINRATIO = 0.70
FOLDS = 3

In [16]:
# inputs
microwave_inputs = ['vv','vh']
optical_inputs = ['red','green','blue','swir','nir', 'ndvi', 'ndwi','nirv']
mixed_inputs =  ['vv_%s'%den for den in optical_inputs] + ['vh_%s'%den for den in optical_inputs] + ['vh_vv']
dynamic_inputs = microwave_inputs + optical_inputs + mixed_inputs
static_inputs = ['slope', 'elevation', 'canopy_height','forest_cover',\
                    'silt', 'sand', 'clay']
all_inputs = static_inputs+dynamic_inputs
inputs = all_inputs

In [17]:
CV = True
fold = None # UNSURE
FOLDS = FOLDS

In [18]:
# drop crops
if DROPCROPS:
    crop_classes = [item[0] for item in lc_dict.items() if item[1] == 'crop']
    dataset = dataset.loc[~dataset['forest_cover(t)'].isin(crop_classes)]

In [19]:
# integer encode forest cover
encoder = LabelEncoder()
dataset = dataset.reindex(sorted(dataset.columns), axis=1) # UNSTABLE, sorting
cols = list(dataset.columns.values)
for col in ['percent(t)','site','date']: # remove some columns
    cols.remove(col)
cols = ['percent(t)','site','date']+cols
dataset = dataset[cols]
dataset['forest_cover(t)'] = encoder.fit_transform(dataset['forest_cover(t)'].values)
for col in dataset.columns:
    if 'forest_cover' in col:
        dataset[col] = dataset['forest_cover(t)']

In [20]:
# normalize features
scaler = MinMaxScaler(feature_range=(0,1))
dataset.replace([np.inf, -np.inf], [1e5, 1e-5], inplace = True)
scaled = scaler.fit_transform(dataset.drop(['site','date'],axis = 1).values)
rescaled = dataset.copy()
rescaled.loc[:,dataset.drop(['site','date'],axis = 1).columns] = scaled
reframed = rescaled.copy()

print('[INFO] Dataset has %d sites'%len(reframed.site.unique()))
reframed.reset_index(drop = True, inplace = True)

[INFO] Dataset has 128 sites


/var/folders/_6/xq5r06kd7qb_g5546y0zvvz80000gn/T/ipykernel_88583/1452303311.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  rescaled.loc[:,dataset.drop(['site','date'],axis = 1).columns] = scaled
/var/folders/_6/xq5r06kd7qb_g5546y0zvvz80000gn/T/ipykernel_88583/1452303311.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  rescaled.loc[:,dataset.drop(['site','date'],axis = 1).columns] = scaled
/var/folders/_6/xq5r06kd7qb_g5546y0zvvz80000gn/T/ipykernel_88583/1452303311.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has

In [21]:
from sklearn.model_selection import KFold

In [22]:
cover = np.sort(reframed['forest_cover(t)'].unique())[0]
sub = reframed.loc[reframed['forest_cover(t)']==cover]
sites = np.sort(sub.site.unique())

In [40]:
np.sort(reframed.site.unique())

array(['ASF Greer', 'Baker Park', 'Balanced Rock', 'Benchmark',
       'Big Indian', 'Big Thicket National Preserve',
       'Bitter Canyon, Castaic', 'Black Cedar', 'Blackberry Hill',
       'Blackstone', 'Blue Park', 'CAF Truchas 1', 'CAF Truchas 2',
       'CNTX_Comal_TX', 'CNTX_Hays_TX', 'Clark Motorway, Malibu',
       'Corralitos New Growth', 'Cottage', 'Cow Mountain', 'D02_Red_Deer',
       'D03_Gibson', 'D03_Willis_Creek', 'D06_Ute Canyon',
       'D07_Cimarron River', 'D10 - Flagstaff Mtn. - Cold',
       'D11_Guanella_Pass', 'D11_Miller_Gulch', 'DUCK CREEK',
       'Deep Creek', 'Deer Hill', 'Double Springs', 'Dubois',
       'Fortynine C2A', 'GNF Lincoln Canyon', 'Gifford',
       'Glendora Ridge, Glendora', 'Grass Mesa', 'Great Divide',
       'HILL_Edwards2_TX', 'HILL_Gillespie_TX', 'HILL_SanSaba_TX',
       'Hall Creek', 'Hammett', 'Hastings Old Growth', 'Jackson',
       'Juanita', 'Keating Cutoff', 'Kuna', 'LNF Cosmic', 'LNF Mayhill',
       'LNF Smokey Bear', 'Lake Hug

In [24]:
KFold(n_splits=3)
list(kf.split(sites))

NameError: name 'kf' is not defined

In [25]:
train_ind=[]

if CV:
    for cover in np.sort(reframed['forest_cover(t)'].unique()):
        sub = reframed.loc[reframed['forest_cover(t)']==cover]
        sites = np.sort(sub.site.unique())
       
        if len(sites)<FOLDS:
            train_sites = sites
        else:
            kf = KFold(n_splits=3)
            train_sites_ind, _ = list(kf.split(sites))[0]
            train_sites = sites[train_sites_ind]

        train_ind+=list(sub.loc[sub.site.isin(train_sites)].index)

In [26]:
train = reframed.loc[train_ind].drop(['site','date'], axis = 1)
test = reframed.loc[~reframed.index.isin(train_ind)].drop(['site','date'], axis = 1)
train.sort_index(inplace = True)
test.sort_index(inplace = True)

train_X, train_y = train.drop(['percent(t)'], axis = 1).values, train['percent(t)'].values
test_X, test_y = test.drop(['percent(t)'], axis = 1).values, test['percent(t)'].values

# reshape input to be 3D [samples, timesteps, features]
if inputs==None: #checksum
    inputs = all_inputs
train_Xr = train_X.reshape((train_X.shape[0], int_lag+1, len(inputs)), order = 'A')
test_Xr = test_X.reshape((test_X.shape[0], int_lag+1, len(inputs)), order = 'A')  

In [41]:
rao_sites = ['ASF Greer', 'Baker Park', 'Balanced Rock', 'Benchmark',
       'Big Indian', 'Big Thicket National Preserve',
       'Bitter Canyon, Castaic', 'Black Cedar', 'Blackberry Hill',
       'Blackstone', 'Blue Park', 'CAF Truchas 1', 'CAF Truchas 2',
       'CNTX_Comal_TX', 'CNTX_Hays_TX', 'Clark Motorway, Malibu',
       'Corralitos New Growth', 'Cottage', 'Cow Mountain', 'D02_Red_Deer',
       'D03_Gibson', 'D03_Willis_Creek', 'D06_Ute Canyon',
       'D07_Cimarron River', 'D10 - Flagstaff Mtn. - Cold',
       'D11_Guanella_Pass', 'D11_Miller_Gulch', 'DUCK CREEK',
       'Deep Creek', 'Deer Hill', 'Double Springs', 'Dubois',
       'Fortynine C2A', 'GNF Lincoln Canyon', 'Gifford',
       'Glendora Ridge, Glendora', 'Grass Mesa', 'Great Divide',
       'HILL_Edwards2_TX', 'HILL_Gillespie_TX', 'HILL_SanSaba_TX',
       'Hall Creek', 'Hammett', 'Hastings Old Growth', 'Jackson',
       'Juanita', 'Keating Cutoff', 'Kuna', 'LNF Cosmic', 'LNF Mayhill',
       'LNF Smokey Bear', 'Lake Hughes', 'Lewis Canyon', 'Lookout Mtn',
       'Lopez Lake', 'Los Alamos', 'Los Robles, Thousand Oaks',
       'Lost Creek, WY', 'Lucky Springs', 'Mad Ridge Fuel Break',
       'Mad River', 'McCloud (SHF)', 'Morgan Creek', 'Mt. Baldy Village',
       'Mt. Woodson Station', 'Muskrat', 'NETX_Wood_TX', 'NOTX_Hood_TX',
       'NOTX_Palo_TX', 'Oak Flat', 'Oak Knoll', 'Ohio Gulch',
       'PNF Cherry', 'PNF White Spar', 'Palisade', 'Panter', 'Pocatello',
       'Pole Creek', 'Price Rec', 'Pulgas', 'Quincy RD', 'RGPL_Dimmit_TX',
       'RMV', 'RUTH', 'Reader Ranch', 'Red Feather', 'Reyes Creek',
       'Rock Spring', 'Rome Overlook', 'Rose Valley', 'Rye Grass Flat',
       'SETX_Newt_TX', 'SETX_SAug_TX', 'San Marcos', 'Sand Creek Desert',
       'Saratoga Summit', 'Sellers Creek', 'Sharpnose', 'Shinar Saddle',
       'Simco', 'Six Shooter', 'Smith Ranch', 'Snowbasin', 'Sonora',
       'South Canyon', 'Spar Canyon', 'Squaw Peak', 'Sugar Hill',
       'Summit2', 'Sunset Heights', 'Sylvan', 'TPEC_Presidio_TX',
       'Table Legs Butte', 'TahoeDonner', 'Tally Lake North',
       'Tapo Canyon, Simi Valley', 'Three Creek',
       'Trippet Ranch, Topanga', 'Tumalo Ridge', 'Tyler Foote',
       'Warm Springs', 'Weed Mill Site', 'Wells', 'West Elk Creek',
       'Ziegler', 'nacimiento', 'ponderosa', 'upper oso']

In [42]:
fang_sites = ['18 Mile', '25 Road Chinquapin', '25 Road manzanita', 'ASF Larson',
       'ASF Little MC', 'ASF Little Pine', 'Adobe Creek', 'Alder',
       'Alder New', 'Ammon', 'Anderson Ridge', 'Angel Creek',
       'Angeles Forest Hwy', 'Apperson Manzanita, Whiteleaf New',
       'Apperson Manzanita, Whiteleaf Old', 'Balanced Rock',
       'Bald Mountain Lookout', 'Banning',
       'Barnes Manzanita, Whiteleaf New',
       'Barnes Manzanita, Whiteleaf Old', 'Barnes Pine, Ponderosa',
       'Basque Hills', 'Beacon Pass', 'Bell Springs', 'Beus', 'Big Creek',
       'Big Creek North', 'Big Thicket National Preserve', 'Big Valley',
       'Bishop/Keoughs Hot Springs', 'Black Canyon', 'Blackstone',
       'Blizzard Gap Bottom', 'Blizzard Gap Middle', 'Blizzard Gap Top',
       'Bloomer Hill - SW Old', 'Blue Mountain', 'Bob Scott', 'Bostetter',
       'Bowman', 'Brandy City Tanoak', 'Bridgeport',
       'Broken Kettle Grasslands - Stevenson Ridge', 'Browns Gap',
       'Buckeye Manzanita, Whiteleaf New',
       'Buckeye Manzanita, Whiteleaf Old', 'Bullwhack', 'Burro Mountian',
       'CNF Alamo Canyon', 'COF Baker', 'COF Blue Ridge', 'COF Cow Trap',
       'COF Mormon Lake', 'Carlin', 'Carson', 'Chester', 'Chiloquin',
       'Circle X, Malibu', 'Clearlake', 'Coffee Point', 'Cold Springs',
       'Coleman', 'Coleman Ponderosa', 'Coleman Sage', 'Combined',
       'Corralitos', 'Corralitos New Growth', 'Corralitos Old Growth',
       'Cottage', 'Cow Creek', 'Crab Creek', 'Crane Flat - Old',
       'Crestview sagebrush combined averages',
       'Crestview sagebrush new growth', 'Crestview sagebrush old growth',
       'Crow 532', 'Cumberland', 'D01_Turquoise_Lake',
       'D02_Ute_Trail_Lower', 'D06_Dry Bluff', 'D06_Ute Canyon',
       'D06_Withers Canyon', 'D07_Cimarron South',
       'D10 - Flagstaff Mtn. - Cold', 'D10 - Jones Hill Upper',
       'D10 - Sledgehammer North', 'D10_Dicks Peak', 'D11_Waterton',
       'D9 Trout Creek', 'DOYLE', 'DUCK CREEK', 'Dad', 'Darling Ridge',
       'Davewood', 'De Sabla', 'Deer Valley - DF', 'Deer Valley-New',
       'Double Springs', 'Dubois', 'Dutch Flat', 'Eagle Lake',
       'Elk Mountain - New Growth', 'Elk Mountain - Old Growth',
       'Elk Mtn', 'Elk Mtn Post Ranch Fire', 'Elko', 'Emerald Bay',
       'Emerald Bay New Growth', 'Emery Creek', 'Esterbrook', 'Evanston',
       'FH-7 - New Growth', 'FH-7 - Old Growth', 'Fencemaker', 'Fernwood',
       'Fish Springs', 'Fishtrap', 'Fort Rock', 'Fortynine C2A',
       'Fouts Springs', 'Fredericks Butte', 'Game Creek',
       'Garfield County Airport', 'Gasquet', 'Gerber',
       'Geysers (Sonoma County)', 'Geysers Peak', 'Grass Valley',
       'Grasshopper', 'Great Divide', 'Greenhorn Yreka', 'Hammett',
       'Happy Camp', 'Haskell Peak Road Manzanita', 'Headquarters',
       'Hebgen Lake', 'Henrys', 'Hitchcock Cedar Fuels', 'Hobble Creek',
       'Hogsback', 'Holst Rd', 'Horse', 'Horse Heaven Hills',
       'Howell Canyon', 'Idaho Falls', 'JAKES VALLEY', 'Jacks Valley',
       'Jackson', 'Janesville', 'Juanita', 'Juniper Creek C2B',
       'Juniper Dunes', 'KNF 429-454', 'KNF Camp 36', 'KNF City',
       'KNF Eagle 5', 'KNF Jacob Lake', 'KNF North', 'KNF South',
       'Kanaka', 'Kawuneeche', 'Keating Cutoff', 'Keeney',
       'Kingsbury Grade', 'Kinsman Ceanothus, Buckbrush New',
       'Kinsman Ceanothus, Buckbrush Old',
       'Kinsman Manzanita, Whiteleaf New',
       'Kinsman Manzanita, Whiteleaf Old', 'Knox', 'Konocti', 'Kuna',
       'Kyle Canyon', 'LHSF Cedar Fuels', 'La Tuna Canyon, Tujunga',
       'Laguna Ridge, Casitas', 'Lake Mendocino', 'Lakeview Ranch',
       'Las Posadas', 'Las Posadas State Forest',
       'Laurel Canyon, Mt. Olympus', 'Lee Vining/June Lake',
       'Lewis Canyon', 'Lincoln Crest', 'Little Beaver',
       'Little Cottonwood', 'Little Mountain 1', 'Long Canyon',
       'Lookout Mtn', 'Los Robles, Thousand Oaks', 'Lucky Springs',
       'Mad Ridge Fuel Break', 'Mad River', 'Main Boulder',
       'Mammoth Airport', 'Maple Canyon', 'Markleeville', 'McCloud',
       'McCoy Ridge', 'Mesa', 'Mill Creek', 'Miranda', 'Monte Cristo',
       'Morgan Creek', 'Mountain Home Ranch', 'Moxa', 'Mt. Baldy Village',
       'Mt. Emma', 'NETX_KAUF_TX', 'NOTX_Clay_TX', 'National',
       'New Harmony', 'New Jungo', 'North Fork', 'Notch Butte', 'Oak Run',
       'Ohio Gulch', 'Old Camptonville manzanita',
       'Old Camptonville tanoak', 'Old Man CG', 'Oregon House', 'PIOCHE',
       'Palisade', 'Palisades', 'Palomino Butte', 'Panther', 'Pardee',
       'Peach Motorway', 'Pebble Beach New Growth',
       'Pebble Beach Old Growth', 'Pickle Gulch', 'Pike County',
       'Pillsbury', 'Pine Acres', 'Pine Creek', 'Pine Valley Pinyon',
       'Pistol Draw', 'Pit River', 'Placerita Canyon', 'Pocatello',
       'Pole Creek', 'Pollock Pines',
       'Ponderosa Basin Ceanothus, Buckbrush New',
       'Ponderosa Basin Ceanothus, Buckbrush Old',
       'Ponderosa Basin Manzanita, Whiteleaf New',
       'Ponderosa Basin Manzanita, Whiteleaf Old', 'Preist', 'Prospect',
       'Pulgas', 'Quartz Hill New Growth', 'Quartz Hill Old Growth',
       'RAINBOW CANYON', 'RUTH', 'Randolph', 'Raspberry', 'Ravendale C2D',
       'Red Cap Manzanita', 'Red Rock', 'Rock Creek', 'Rock Spring',
       'Rollins Reservoir', 'Rome Overlook', 'Rye Grass Flat',
       'SCE Hwy 74', 'SOPL_Crane_TX', 'Saddle Mountains',
       'Saginaw Pine, Ponderosa', 'Sand Creek Desert', 'Sawmill 200',
       'Schueren Road, Malibu', 'Sedge', 'Sellers Creek', 'Sharpnose',
       'Shasta Dam', 'Shingletown', 'Shirttail', 'Shoshone Basin',
       'Silver Fork', 'Simco', 'Sisar Canyon, Upper Ojai Valley',
       'Slate Creek', 'Sledgehammer South', 'Smiths Fork',
       'Snow Springs 1', 'South Canyon', 'South Grays Creek',
       'Spar Canyon', 'Spooner', 'Spring Creek', 'Spunky Canyon',
       'Square Top', 'Squaw Peak', 'Station 14 New Growth',
       'Station 14 Old Growth and 1000 Hr', 'Station 60 New Growth',
       'Station 60 old growth and 1000 Hr.', 'Steck Park',
       'Stunt Road, Calabasas', 'Sublett', 'Sugar Hill', 'Summit1',
       'Summit2', 'Sunset Heights', 'Sweetland',
       'Sweetwater Campground 2', 'Switchback', 'TPEC_Presidio_TX',
       'Table Legs Butte', 'Tapo Canyon, Simi Valley', 'Taylor Butte',
       'Telford', 'Templin Highway', 'Three Creek', 'Three Island',
       'Throckmorton', 'Thunder Basin Work Center', 'Trough - New Growth',
       'Trough - Old Growth', 'Trumbull', 'Twin Lakes', 'Tyler Foote',
       'USBOR', 'Upper Ojai Valley', 'Walker Ridge', 'Warm Springs',
       'Waubonsie Cedar Fuels', 'Weaverville New Growth',
       'Weaverville Old Growth', 'Weed Mill Site', 'Wells',
       'West Bridger', 'West Elk Creek', 'Whaleback Mtn', 'Whitetail',
       'Woolsey Canyon', 'Ziegler']

In [45]:
len(list(set(rao_sites) & set(fang_sites)))

54

In [ ]:
len(list(set(rao_sites) & set(fang_sites)))

In [52]:
len(rao_sites)

128

In [53]:
len(fang_sites)

335

In [58]:
rao_sites = [site.lower() for site in rao_sites]
fang_sites = [site.lower() for site in fang_sites]

In [59]:
count = 0
for site in rao_sites:
    if site in fang_sites:
        count += 1

count

54